In [1]:
###UPF Brain structure brain project University of Helsinki 6.02.2024
###Arsene Kanyamibwa, Daniel Fängström 
rm(list=ls())

#install packages (if needed)
#install.packages("tidyverse")
#install.packages("ggplot2")
#install.packages("Hmisc")

#:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
#Load the necessary libraries 
  library(tidyverse)
  library(ggplot2)
  library(ggpubr)
  library(Hmisc)
  library(readxl)
 
#:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::


── Attaching core tidyverse packages ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘Hmisc’


The following objects are masked from ‘package:dplyr’:

    src, summarize


The following objects

In [4]:
UKBB_participant_diet_answers=read.table('/dagher/dagher11/filip/UPF/data/coalesced_intake.csv', sep=',', quote='"', header=T)
colnames(UKBB_participant_diet_answers)=gsub('_coalesced','',colnames(UKBB_participant_diet_answers)) #remove coalesced from column names
UKBB_participant_diet_answers$Participant=UKBB_participant_diet_answers$data.eid
mastersheet=readRDS('/dagher/dagher11/filip/UPF/data/mastersheet_clean.rds')

# MANUALLY change column names to correspond to the mastersheet

In [4]:
change_to=c('pure_fruit_vegetable_juice_intake',
'sugar_added_to_coffee_intake',
'sugar_added_to_tea_intake',
'beer/cider_intake',
'whole-wheat_cereal_intake',
"goat's_cheese_intake",
'number_of_bread_slices_with_butter/margarine_intake',
'number_of_baguettes_with_butter/margarine_intake',
'number_of_baps_with_butter/margarine_intake',
'number_of_bread_rolls_with_butter/margarine_intake',
'number_of_crackers/crispbreads_with_butter/margarine_intake',
'number_of_oatcakes_with_butter/margarine_intake',
'number_of_other_bread_types_with_butter/margarine_intake',
'ice-cream_intake',
'milk-based_pudding_intake',
'other_milk-based_pudding_intake',
'chocolate-covered_raisin_intake',
'chocolate-covered_biscuits_intake',
'powdered/instant_soup_intake',
'crumbed_or_deep-fried_poultry_intake',
'lobster/crab_intake',
'vegetarian_sausages/burgers_intake',
'boiled/baked_potatoes_intake',
'cabbage/kale_intake',
'turnip/swede_intake',
'peach/nectarine_intake',
'other_coffee_type_intake')

to_change=c('pure_fruitvegetable_juice_intake','intake_of_sugar_added_to_coffee','intake_of_sugar_added_to_tea',
            'beercider_intake','whole.wheat_cereal_intake','goats_cheese_intake', 'number_of_bread_slices_with_buttermargarine',
            'number_of_baguettes_with_buttermargarine','number_of_baps_with__buttermargarine',
            'number_of_bread_rolls_with__buttermargarine','number_of_crackerscrispbreads_with_buttermargarine',
            'number_of_oatcakes_with__buttermargarine','number_of_other_bread_types_with__buttermargarine',
            'ice.cream_intake','milk.based_pudding_intake','other_milk.based_pudding_intake','chocolate.covered_raisin_intake',
            'chocolate.covered_biscuits_intake','powderedinstant_soup_intake','crumbed_or_deep.fried_poultry_intake',
            'lobstercrab_intake','vegetarian_sausagesburgers_intake','boiledbaked_potatoes_intake','cabbagekale_intake',
            'turnipswede_intake','peachnectarine_intake','other_coffee_type')

for (i in 1:length(change_to)){

    names(UKBB_participant_diet_answers)[names(UKBB_participant_diet_answers) == to_change[i]] <- change_to[i]
    
}

In [5]:
# Function to calculate consumption for each participant
calculate_consumption <- function(response, serving_size, energy_per_100g_or_ml) {
  # Calculate total amount consumed per item by participant
  total_consumed_g_or_ml <- response * serving_size
  
  # Calculate energy per serving
  energy_per_serving_KJ <- serving_size * (energy_per_100g_or_ml/100)
  
  # Calculate energy intake for each participant by item
  total_energy_consumed <- energy_per_serving_KJ * response
  
  # Return total amount consumed and energy per serving
  return(list(total_consumed_g_or_ml = total_consumed_g_or_ml, energy_per_serving_KJ = energy_per_serving_KJ, total_energy_consumed = total_energy_consumed))
}

In [6]:
final_UKBB_results_table <- data.frame(participant=vector("character", length(UKBB_participant_diet_answers$Participant)))
names=matrix(ncol=1,nrow=nrow(mastersheet))

pb = txtProgressBar(min = 0, max = nrow(UKBB_participant_diet_answers), initial = 0) 

# Loop through each row in UKBB_participant_diet_asnwers

for (i in 1:nrow(mastersheet)) {  #look into all rows of the dataframe
  current_item_name <- mastersheet[i,2] #look into item names and create a variable
  current_serving_g <- mastersheet[i,8] #look serving size in grams column names and create a variable
  current_serving_ml <- mastersheet[i,9] #look serving size in ml column and create a variable
  current_Energy <- mastersheet[i,10] #look energy in 100 ml or grams column and create a variable
  if (is.na(current_serving_g)){
    current_unit <- current_serving_ml
  }  else {
      current_unit <- current_serving_g #check if item has a NA value for serving size in ml or gr if not use the column with a value
  }
  
  current_item_name <- gsub(" ","_",current_item_name, fixed = TRUE) #eliminate space from item names
  current_item_name_intake <- paste(current_item_name, "_intake") #add intake at the end of item names
  current_item_name_intake <- gsub(" ","",current_item_name_intake, fixed = TRUE) #too much of cognitive power for 3 lines
  current_item_name_intake <- tolower(current_item_name_intake) #we made the initial letter non capital
  column_item_total_consumed_g_or_ml <-  paste(current_item_name,"_total_consumed_g_or_ml", sep = "")
  final_UKBB_results_table[, column_item_total_consumed_g_or_ml] <- NA
  column_item_energy_per_serving_KJ <-  paste(current_item_name,"_energy_per_serving_KJ", sep = "")
  final_UKBB_results_table[, column_item_energy_per_serving_KJ] <- NA
  column_item_total_energy_consumed <-  paste(current_item_name,"_total_energy_consumed", sep = "")
  final_UKBB_results_table[, column_item_total_energy_consumed] <- NA
  
  for (j in 1:nrow(UKBB_participant_diet_answers)){
    Participant <- as.character(UKBB_participant_diet_answers$Participant[j]) #Go through dataframe and find participant numbers final_UKBB_results_table[j, "participant"] <- as.character(UKBB_participant_diet_answers$Participant[j])
    #print(Participant)
    if (is.null(final_UKBB_results_table[j, Participant])){  #m
      final_UKBB_results_table[j, "participant"] <- Participant  #take participant variable value and insert in empty dataframe
    }
    
    indx <- grep(paste('^',current_item_name_intake, sep=''), colnames(UKBB_participant_diet_answers))  #change from grepl to grep to give us columns number instead of logical index
    #print(UKBB_participant_diet_answers[j,indx]) #then prints the value of dataframe in j= participant and column found
    #print(current_item_name_intake)
    response <- UKBB_participant_diet_answers[j,indx] #making it a variable
    setTxtProgressBar(pb,j)
    if (!length(indx)==0){  # Added this - sometimes current_item_name_intake is not found (because there is not _intake at the end), so this needs a new condition 
      if (!is.na(response) ) { #If the response is not NA, go through the rest of the logic
          if (is.character(response)) { # If the response is  character, check what type and reformat it
            #print(endsWith(response,"+"))
            if (endsWith(response,"+")) {
              response <- gsub("+", "", response, fixed = TRUE)
              #print(response)
            } else {
              response <- gsub("varied", NA, response, fixed = TRUE) #the number remains a string
              response <- gsub("half", "0.5", response, fixed = TRUE) #the number remains a string
              response <- gsub("quarter", "0.25", response, fixed = TRUE) #the number remains a string

            }
            #print(response)
              response <- ifelse(is.na(response), NA, as.numeric(response)) #making sure it is a number not string. 0.5 is not included as an integer 
            #print(response)
          }
          UKBB_total_consumption <- calculate_consumption(response,current_unit,current_Energy)
          final_UKBB_results_table[j, column_item_energy_per_serving_KJ] <- UKBB_total_consumption$energy_per_serving_KJ
          final_UKBB_results_table[j,column_item_total_energy_consumed] <-  UKBB_total_consumption$total_energy_consumed
          final_UKBB_results_table[j, column_item_total_consumed_g_or_ml] <- UKBB_total_consumption$total_consumed_g_or_ml
          #print(final_UKBB_results_table)
        }
    }
      #else{names[i]=current_item_name_intake} # check which variables from mastersheet are still missing
  }

    close(pb)
        
}



    # Store results
write.table(final_UKBB_results_table,'/dagher/dagher11/filip/UPF/data/Participants_consumption.csv', quote=T, row.names=F)



=                                                                               ========================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [2]:
final_UKBB_results_table=read.table('/dagher/dagher11/filip/UPF/data/Participants_consumption.csv', header=T, sep=' ')

# NOVA calculations

In [5]:
#:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
#Calculations for total energy and NOVA precentage

#Get column indices that contain "participant" or "_total_energy_consumed"
columns_to_keep <- grepl("participant|_total_energy_consumed", colnames(final_UKBB_results_table))

# Keep only the columns containing "participant" or "_total_energy_consumed"
final_UKBB_total_consumption <- final_UKBB_results_table[, columns_to_keep]

#check dataframe
#str(final_UKBB_total_consumption)

#same procedure without trasposing first.
# Get the column names of the dataframe
col_names <- colnames(final_UKBB_total_consumption)
col_names <- gsub("_total_energy_consumed", "", col_names)
col_names <- gsub("_", " ", col_names)
# Assign the modified column names back to the dataframe
colnames(final_UKBB_total_consumption) <- col_names
#change to long format
final_UKBB_total_consumption <- gather(final_UKBB_total_consumption, 
                                       key = "Food_Item",
                                       value = "Energy_KJ", -1)

#Calculate total energy (i originally calculated the energy and wanted to add the NOVA but I tried another approach)
#final_UKBB_total_consumption <- final_UKBB_total_consumption %>%
#  mutate(Total_energy = rowSums(select(.,-1), na.rm=TRUE))

#load another dataframe to insert nova groups and merge
#Store results
#getwd()
#setwd("P:/h345/obrain_labspace/Projects/PhD_projects/MARVEL/MARVEL III_UPF/03_Experiment/03_Data")
#write.csv(final_UKBB_total_consumption, file = "Final_UKBB_total_consumption.csv")
#I transposed the document in excel just for the sake of speed, I was getting a little frustarted with the errors 24.04.2024

#load new document
final_UKBB_total_consumption2 <- read.csv("/dagher/dagher11/filip/UPF/data/Final_UKBB_total_consumption_v2.csv")

# Get the column names of the dataframe
col_names <- colnames(final_UKBB_total_consumption2)

# Remove "X" from the beginning of column names (if present)
col_names <- gsub("^X", "", col_names)

# Assign the modified column names back to the dataframe
colnames(final_UKBB_total_consumption2) <- col_names
#remove the end with total_energy_consumed
for (col in colnames(final_UKBB_total_consumption2)) {
  final_UKBB_total_consumption2[[col]] <- gsub("_total_energy_consumed", "", final_UKBB_total_consumption2[[col]])
}
# Remove "_" from values in the column named "Food_item"
final_UKBB_total_consumption2$Food_item <- gsub("_", " ", final_UKBB_total_consumption2$Food_item)
# Find matching values and extract corresponding NOVA values
final_UKBB_total_consumption2$NOVA <- mastersheet$`NOVA _code`[match(final_UKBB_total_consumption2$Food_item, mastersheet$Item_Name)]
#make NOVA column factor
final_UKBB_total_consumption2$NOVA <- as.factor(final_UKBB_total_consumption2$NOVA)
#remove columns in position 2 and 11
final_UKBB_total_consumption2 <- final_UKBB_total_consumption2 %>% 
  select(-2,-3,-4,-5,-6,-7,-8,-9,-10,-11) 
#Match names on food items
names(final_UKBB_total_consumption2)[1]= "Food_Item"
#merge two dataframes
main_df <- merge(final_UKBB_total_consumption, final_UKBB_total_consumption2, by= "Food_Item", all.x = TRUE)

#I tried this way because the most obvious ones did not work (see like 197)
summary_df <- aggregate(main_df$Energy_KJ, list(main_df$participant, main_df$NOVA), FUN=sum, na.rm = TRUE) 
#changed names
{
  names(summary_df)[1] = "Participant"
  names(summary_df)[2] = "NOVA_Group"
  names(summary_df)[3] = "Total_Energy"
  }

#print(summary_df)

# Step 2: Summarize the energy consumed per NOVA group
# Group the dataframe by participant ID and NOVA group, and calculate the sum of energy consumed
summary_df <- main_df %>%
  group_by(participant, NOVA) %>%
  summarise(total_energy_consumed = sum(Energy_KJ, na.rm=T)) #for some reason this did not work
#this error kept coming and I could not solve it
#Error in summarize(., Total_energy = sum(Energy_KJ)) : argument "by" is missing, with no default


`summarise()` has grouped output by 'participant'. You can override using the `.groups` argument.


In [6]:
write.table(summary_df,'/dagher/dagher11/filip/UPF/data/Participants_summary_NOVA.csv', quote=T, row.names=F)


In [8]:
summary_df_nona=na.omit(summary_df)

In [9]:
NOVA_energy=pivot_wider(
  summary_df_nona,
  id_cols = c(participant),
  names_from = NOVA,
  values_from = total_energy_consumed,
  names_prefix = "kJ_NOVA_"
)

In [10]:
write.table(NOVA_energy,'/dagher/dagher11/filip/UPF/data/NOVA_energy_values_wide.csv', quote=T, row.names=F)
